# Example of a schedview event timeline

In [ ]:
night = '2024-12-08'
telescope = 'Simonyi'
visit_origin = "lsstcomcam"

In [ ]:
# Use more recent versions of rubin_scheduler and schedview (will become unnecessary soon)
import sys
import os
import astropy.utils.iers
import bokeh.io

# Tell astropy we don't need super high precision:
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Tell rubin_scheduler where to find its data
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Prepare jupyter to show bokeh plots
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
devel = False
if devel:
    %load_ext lab_black
    %load_ext autoreload
    %autoreload 1

    extra_paths = (
        "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler",
        "/sdf/data/rubin/user/neilsen/devel/rubin_sim",
        "/sdf/data/rubin/user/neilsen/devel/schedview",
    )
    for extra_path in extra_paths:
        sys.path.insert(0, extra_path)

    %aimport schedview.dayobs
    %aimport schedview.collect.nightreport
    %aimport schedview.collect.visits
    %aimport schedview.collect.efd
    %aimport schedview.compute.astro
    %aimport schedview.compute.obsblocks
    %aimport schedview.compute.nightreport
    %aimport schedview.plot.timeline
else:
    extra_paths = (
        "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.4.1.dev2+g9f70dc6.d20241204",
        "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.1.1.dev71+g0a9c0ea9",
        "/sdf/data/rubin/shared/scheduler/packages/SP-1789/schedview",
    )
    for extra_path in extra_paths:
        sys.path.insert(0, extra_path)

    import schedview.dayobs
    import schedview.collect.nightreport
    import schedview.collect.visits
    import schedview.collect.efd
    import schedview.compute.astro
    import schedview.compute.obsblocks
    import schedview.compute.nightreport
    import schedview.plot.timeline

In [ ]:
day_obs = schedview.dayobs.DayObs.from_date(night)

In [ ]:
visits = schedview.collect.visits.read_visits(
    day_obs, visit_origin, stackers=schedview.collect.visits.NIGHT_STACKERS
)

In [ ]:
messages = schedview.collect.nightreport.get_night_narrative(day_obs, telescope)
messages_with_duration = [m for m in messages if m["date_begin"] < m["date_end"]]

In [ ]:
new_messages = []
for message in messages:
    time_lost_type = (
        "" if message["time_lost_type"] == "None" else message["time_lost_type"]
    )
    writer = "Human" if message["is_human"] else "Automated "
    for component in message["components"]:
        new_message = {"factor": f"{writer} log message ({component} {time_lost_type})"}
        new_message.update(message)
        new_messages.append(new_message)

In [ ]:
sal_indexes = schedview.collect.efd.SAL_INDEX_GUESSES[visit_origin]

In [ ]:
scheduler_dependencies = await schedview.collect.efd.query_efd_topic_for_night(
    "lsst.sal.Scheduler.logevent_dependenciesVersions", day_obs, sal_indexes
)

In [ ]:
scheduler_configuration = await schedview.collect.efd.query_efd_topic_for_night(
    "lsst.sal.Scheduler.logevent_configurationApplied", day_obs, sal_indexes
)

In [ ]:
block_status = await schedview.collect.efd.query_efd_topic_for_night(
    "lsst.sal.Scheduler.logevent_blockStatus", day_obs, sal_indexes
)
block_spans = schedview.compute.obsblocks.compute_block_spans(block_status)

In [ ]:
scheduler_snapshots = await schedview.collect.efd.query_efd_topic_for_night(
    "lsst.sal.Scheduler.logevent_largeFileObjectAvailable", day_obs, sal_indexes
)

In [ ]:
night_events = schedview.compute.astro.night_events(day_obs.date)

In [ ]:
median_model_sky = schedview.compute.astro.get_median_model_sky(day_obs)

In [ ]:
p = schedview.plot.timeline.make_timeline_scatterplots(
    visits=visits,
    sun=night_events,
    model_sky=median_model_sky,
    visit_timeline=visits,
    block_spans=block_spans,
    block_status=block_status,
    scheduler_snapshots=scheduler_snapshots,
    scheduler_dependencies=scheduler_dependencies,
    scheduler_configuration=scheduler_configuration,
    log_messages=messages,
    message_spans=messages_with_duration,
)

bokeh.io.show(p)

You can make a full stand-alone (*not* using jupyter or Times Square) dashboard web application using `panel` like this:

In [ ]:
import inspect
from schedview.examples.app import event_timeline

print(inspect.getsource(event_timeline))